<img src="https://raw.githubusercontent.com/brazil-data-cube/code-gallery/master/img/logo-bdc.png" align="right" width="64"/>

# <span style="color:#336699">Introduction to the SpatioTemporal Asset Catalog (STAC) static on AWS</span>
<hr style="border:2px solid #0077b9;">

<div style="text-align: left;">
    <a href="https://nbviewer.jupyter.org/github/brazil-data-cube/code-gallery/blob/master/jupyter/Python/stac/stac-introduction.ipynb"><img src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg" align="center"/></a>
</div>

<br/>

<div style="text-align: center;font-size: 90%;">
    Matheus Zaglia<sup><a href="https://orcid.org/0000-0001-6181-2158"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Rennan Marujo<sup><a href="https://orcid.org/0000-0002-0082-9498"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Gilberto R. Queiroz<sup><a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:brazildatacube@inpe.br">brazildatacube@inpe.br</a>
    <br/><br/>
    Last Update: December 08, 2021
</div>

<br/>

<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;">
<b>Abstract.</b> This Jupyter Notebook gives an overview on how to use the AWS Static STAC service to discover and access the data products from the <em>Brazil Data Cube</em> on AWS OpenData Program.
</div>    

<br/>

# AWS STAC Catalogs
<hr style="border:1px solid #0077b9;">

<p>For Sentinel-2 EO Data Cubes, use <a href="https://bdc-sentinel-2.s3.us-west-2.amazonaws.com/catalog.json">https://bdc-sentinel-2.s3.us-west-2.amazonaws.com/catalog.json</a></p>
<p>For CBERS EO Data Cubes, use <a href="https://bdc-cbers.s3.us-west-2.amazonaws.com/catalog.json">https://bdc-cbers.s3.us-west-2.amazonaws.com/catalog.json</a></p>

In this notebook, we are going to use `Sentinel-2`.


# STAC Client API
<hr style="border:1px solid #0077b9;">

For running the examples in this Jupyter Notebook you will need to install the [pystac-client](https://pystac-client.readthedocs.io/en/latest/). To install it from PyPI using `pip`, use the following command:

In [ ]:
!pip install pystac-client==0.3.2

In [ ]:
!pip install rasterio

In order to access the funcionalities of the client API, you should import the `pystac_client` package, as follows:

In [ ]:
import pystac_client

After that, you can check the installed `pystac_client` package version:

In [ ]:
pystac_client.__version__

Then, create a `pystac_client.Client` object attached to the Brazil Data Cube' STAC service:

In [ ]:
service = pystac_client.Client.open('https://bdc-sentinel-2.s3.us-west-2.amazonaws.com/catalog.json')
service

# Listing the Available Data Products
<hr style="border:1px solid #0077b9;">

In the Jupyter environment, the `STAC` object will list the available image and data cube collections from the service:

In [ ]:
for collection in service.get_collections():
    print(collection)

## Retrieving the Metadata of a Data Product
<hr style="border:1px solid #0077b9;">

The `collection` method returns information about a given image or data cube collection identified by its name. In this example we are retrieving information about the datacube collection `S2-16D-1`:

In [ ]:
collection = service.get_collection('S2-16D-1')
collection.to_dict()

## Retrieving Collection Items
<hr style="border:1px solid #0077b9;">

Since we are using a `static` catalog, you cannot query for items using spatial filter. Instead, you must either traverse entire catalog using `get_all_items()` or query by identifier `.get_item(itemId, recursive=True)`. Keep in mind that it will traverse the entire catalog and may took too long time to finish. We recommend you to use [`stac-browser`](https://radiantearth.github.io/stac-browser/#/external/bdc-sentinel-2.s3.us-west-2.amazonaws.com/catalog.json) to navigate to the catalog and then look for the region of interest.
In the below example, we are going to use the following `s2-16d/v1/079/085/2018/09` which represents an region on Brazilian Amazon Biome.


In [ ]:
# Display an item only
catalog_201809 = pystac_client.Client.open('https://bdc-sentinel-2.s3.us-west-2.amazonaws.com/s2-16d/v1/079/085/2018/09/catalog.json')

for item in catalog_201809.get_all_items():
    print(item)

In [ ]:
items = list(catalog_201809.get_all_items())
item = items[0]
item

## Assets
<hr style="border:1px solid #0077b9;">

The assets with the links to the images, thumbnails or specific metadata files, can be accessed through the property `assets` (from a given item):

In [ ]:
assets = item.assets

Then, from the assets it is possible to traverse or access individual elements:

In [ ]:
for k in assets.keys():
    print(k)

The metadata related to the Sentinel 2A/2B blue band is available under the dictionary key `B02`:

In [ ]:
blue_asset = assets['B02']
blue_asset

To iterate in the item's assets, use the following pattern:

In [ ]:
for asset in assets.values():
    print(asset)

## Using RasterIO and NumPy
<hr style="border:1px solid #0077b9;">

The `rasterio` library can be used to read image files from the Brazil Data Cube' service on-the-fly and then to create `NumPy` arrays. The `read` method of an `Item` can be used to perform the reading and array creation:

In [ ]:
import rasterio

In [ ]:
with rasterio.open(assets['B08'].href) as nir_ds:
    nir = nir_ds.read(1)

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>Note:</b> If there are errors because of your pyproj version, you can run the code below as specified in <a  href="https://rasterio.readthedocs.io/en/latest/faq.html#why-can-t-rasterio-find-proj-db-rasterio-from-pypi-versions-1-2-0" target="_blank">rasterio documentation</a> and try again:

       import os
       del os.environ['PROJ_LIB']
</div>

In [ ]:
nir

The next cell code import the `Window` class from the `rasterio` library in order to retrieve a subset of an image and then create an array:

In [ ]:
from rasterio.windows import Window

We have prepared a basic function `read()`to read raster windows as [`numpy.ma.masked_array`](https://numpy.org/doc/stable/reference/maskedarray.generic.html).

In [ ]:
def read(uri: str, window: Window, masked: bool = True):
    """Read raster window as numpy.ma.masked_array."""
    with rasterio.open(uri) as ds:
        return ds.read(1, window=window, masked=masked)

We can specify a subset of the image file (window or chunck) to be read. Let's read a range that starts on pixel (0, 0) with 500 x 500 and column 0 to column 500, for the spectral bands `red`, `green` and `blue`:

In [ ]:
red = read(assets['B04'].href, window=Window(0, 0, 500, 500)) # Window(col_off, row_off, width, height)

In [ ]:
green = read(assets['B03'].href, window=Window(0, 0, 500, 500))

In [ ]:
blue = read(assets['B02'].href, window=Window(0, 0, 500, 500))

In [ ]:
blue

## Using Matplotlib to Visualize Images
<hr style="border:1px solid #0077b9;">

The `Matplotlib` cab be used to plot the arrays read in the last section:

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(12, 4))
ax1.imshow(red, cmap='gray')
ax2.imshow(green, cmap='gray')
ax3.imshow(blue, cmap='gray')

Using `Numpy` we can stack the previous arrays and use `Matplotlib` to plot a color image, but first we need to normalize their values:

In [ ]:
import numpy

In [ ]:
def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

In [ ]:
rgb = numpy.dstack((normalize(red), normalize(green), normalize(blue)))
plt.imshow(rgb)

# Retrieving Image Files
<hr style="border:1px solid #0077b9;">

The file related to an asset can be retrieved through the `download` method. The cell code below shows ho to download the image file associated to the asset into a folder named `img`:

In [ ]:
import os
from urllib.parse import urlparse

import requests
from pystac import Asset
from tqdm import tqdm

def download(asset: Asset, directory: str = None, chunk_size: int = 1024 * 16, **request_options) -> str:
    """Smart download STAC Item asset.
    
    This method uses a checksum validation and a progress bar to monitor download status.
    """
    if directory is None:
        directory = ''

    response = requests.get(asset.href, stream=True, **request_options)
    output_file = os.path.join(directory, urlparse(asset.href)[2].split('/')[-1])
    os.makedirs(directory, exist_ok=True)
    total_bytes = int(response.headers.get('content-length', 0))
    with tqdm.wrapattr(open(output_file, 'wb'), 'write', miniters=1, total=total_bytes, desc=os.path.basename(output_file)) as fout:
        for chunk in response.iter_content(chunk_size=chunk_size):
            fout.write(chunk)

In [ ]:
download(assets['B08'], 'img')

In order to download all files related to an item, iterate over assets and download each one as following:

In [ ]:
for asset in assets.values():
    download(asset, 'images')

Note that the URL for a given asset can be retrieved by the property `href`:

In [ ]:
blue_asset.href

# References
<hr style="border:1px solid #0077b9;">

- [Spatio Temporal Asset Catalog Specification](https://stacspec.org/)


- [Python Client Library for STAC Service](https://pystac-client.readthedocs.io/en/latest/)

# See also the following Jupyter Notebooks
<hr style="border:1px solid #0077b9;">

* [Image processing on images obtained through STAC](./stac-image-processing.ipynb)


* [NDVI calculation on images obtained through STAC](./stac-introduction.ipynb)